In [33]:
import os
from betfairlightweight import APIClient, filters
uname = os.environ['betfair_username']
pword = os.environ['betfair_password']
key = os.environ['betfair_key']
bf = APIClient(uname, pword, app_key=key,
               certs='/Users/jonaspaulwestermann/Dev/marchmadness/certs',
               locale='spain', lightweight=True)
bf.login()

{'loginStatus': 'SUCCESS',
 'sessionToken': 'XAakl7HJcBVDHRP5QVZ8TI26fuko9yf3TP53M+LX7yA='}

In [25]:
bf.betting.list_competitions()

[{'competition': {'id': '10526369', 'name': 'Eurocup Men'},
  'competitionRegion': 'International',
  'marketCount': 4},
 {'competition': {'id': '175680', 'name': 'Turkish 1 Lig'},
  'competitionRegion': 'TUR',
  'marketCount': 27},
 {'competition': {'id': '10526250', 'name': 'FIBA Champions League'},
  'competitionRegion': 'International',
  'marketCount': 2},
 {'competition': {'id': '856134', 'name': 'Colombian Primera B'},
  'competitionRegion': 'COL',
  'marketCount': 18},
 {'competition': {'id': '139', 'name': 'Ukrainian Premier League'},
  'competitionRegion': 'UKR',
  'marketCount': 9},
 {'competition': {'id': '8580745', 'name': 'Israeli Liga Bet - South A'},
  'competitionRegion': 'ISR',
  'marketCount': 9},
 {'competition': {'id': '15', 'name': 'Bulgarian A League'},
  'competitionRegion': 'BGR',
  'marketCount': 9},
 {'competition': {'id': '309972', 'name': 'AFC Cup'},
  'competitionRegion': 'International',
  'marketCount': 45},
 {'competition': {'id': '81', 'name': 'Italian